# Modelagem do problema do Titanic

 - Realização do tratamento de outliers (se necessário); 
 - Criação prática das features (se necessário);
 - Normalização; (perguntar)
 - Execução do Encoder;
 - Treinamento, Validação e Teste do modelo;

##### Importando as bibliotecas necessárias e dataset processado

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Carregando o modelo processado na etapa anterior (pré processamento)
model_path = r'C:\Users\Fernando Costa\Desktop\Arquivos Lucas\DS\Projetos GITHUB\Titanic\df_model.parquet'
df_model = pd.read_parquet(model_path)

# Definindo as colunas categóricas do dataset
col_categoricas = ['Pclass', 'Sex', 'Cabin', 'Embarked']

## Outliers e Novas Features

Por conta da limitação desta base de dados, apenas será utilizada a nova feature implementada na etapa de Pre Processamento e os outliers ignorados.

## Segregando Dados

Implementando a segregação dos dados de treino e dados de teste

In [2]:
X = df_model.drop(columns='Survived')  # isolando a variável alvo do restante
y = df_model['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Encoding

Importando e segregando dados para utilização do encoder

In [3]:
# Inicializar o encoder
catboost_encoder = CatBoostEncoder(cols = col_categoricas, random_state = 42) # Utilizar sempre um random_state para garantir replicabilidade

# Codificando apenas as colunas categóricas 'Pclass', 'Sex', 'Cabin', 'Embarked'
#X_train2encode = X_train
#y_train2encode = y_train

# Ajustar o encoder nos dados de treino
catboost_encoder.fit(X_train, y_train)

CatBoostEncoder(cols=['Pclass', 'Sex', 'Cabin', 'Embarked'], random_state=42)

Transformando os dados de treino e teste

In [4]:
X_train.head() # Visualizando dados antes da aplicação do encoder

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Taxa_x_SibSp
331,1,male,45.5,0,0,28.5000,C124,S,0.0000
733,2,male,23.0,0,0,13.0000,NA,S,0.0000
382,3,male,32.0,0,0,7.9250,NA,S,0.0000
704,3,male,26.0,1,0,7.8542,NA,S,7.8542
813,3,female,6.0,4,2,31.2750,NA,S,125.1000


In [5]:
# Aplicando a transformação com o encoder treinado na base de treino e de teste
X_train = catboost_encoder.transform(X_train) 
X_test = catboost_encoder.transform(X_test)

In [6]:
X_train.head() # Visualizando os dados após aplicação do encoder

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Taxa_x_SibSp
331,0.605954,0.186702,45.5,0,0,28.5000,0.125468,0.335316,0.0000
733,0.482740,0.186702,23.0,0,0,13.0000,0.298513,0.335316,0.0000
382,0.241545,0.186702,32.0,0,0,7.9250,0.298513,0.335316,0.0000
704,0.241545,0.186702,26.0,1,0,7.8542,0.298513,0.335316,7.8542
813,0.241545,0.737302,6.0,4,2,31.2750,0.298513,0.335316,125.1000


## Treinamento, Validação e Teste do Modelo

Para fins de treinamento, utilizaremos dois algoritmos: RandomForest e Regressão Logística.

In [7]:
# Inicializando e treinando o modelo RandomForest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Inicializando e treinando o modelo de Regressão Logística
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

c:\Users\Fernando Costa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

Para Validar os modelos treinados, utilizaremos a métrica roc-auc

In [8]:
# Fazendo as previsões
rf_predictions = rf_model.predict_proba(X_test)[:, 1] # Predict proba fornece a probabilidade do item ser ou não positivo (0.6 por exemplo), enquanto o predict fornecerá 0 ou 1 apenas
lr_predictions = lr_model.predict_proba(X_test)[:, 1]
#lr_predictions = lr_model.predic

# Avaliando o desempenho com ROC-AUC
rf_auc = roc_auc_score(y_test, rf_predictions) # search, search tradeoff de precisão e recall
lr_auc = roc_auc_score(y_test, lr_predictions)

print(f'Random Forest ROC-AUC: {rf_auc:.4f}')
print(f'Regressão Logística ROC-AUC: {lr_auc:.4f}')

Random Forest ROC-AUC: 0.8741
Regressão Logística ROC-AUC: 0.8699


## Testando Modelos na "Prática"

Importando a base de dados de "Teste real", aplicando o processamento realizado anteriormente ao df_model

In [9]:
# Importando a função criada a partir do pré processamento:
from PreProcessing_titanic import PP_titanic_function

aval_path = r'C:\Users\Fernando Costa\Desktop\Arquivos Lucas\DS\Projetos GITHUB\Titanic\Base_dados\test.csv'
X_aval_original = pd.read_csv(aval_path)
X_aval = X_aval_original.copy()

In [10]:
# Breve teste do df de avaliação. Checando valores nulos
display(X_aval.info())
display(X_aval.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


None

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [11]:
# Aplicando o processamento ao novo df
X_aval = PP_titanic_function(X_aval)

c:\Users\Fernando Costa\Desktop\Arquivos Lucas\DS\Projetos GITHUB\Titanic\PreProcessing_titanic.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model[col].fillna(-1, inplace=True)
c:\Users\Fernando Costa\Desktop\Arquivos Lucas\DS\Projetos GITHUB\Titanic\PreProcessing_titanic.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on 

In [12]:
# Verificando se funcionou como deveria o tratamento
display(X_aval.info())
display(X_aval.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        418 non-null    object 
 1   Sex           418 non-null    object 
 2   Age           418 non-null    float64
 3   SibSp         418 non-null    int64  
 4   Parch         418 non-null    int64  
 5   Fare          418 non-null    float64
 6   Cabin         418 non-null    object 
 7   Embarked      418 non-null    object 
 8   Taxa_x_SibSp  418 non-null    float64
dtypes: float64(3), int64(2), object(4)
memory usage: 29.5+ KB


None

Pclass          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            0
Cabin           0
Embarked        0
Taxa_x_SibSp    0
dtype: int64

Utilizando o Encoder ao df de avaliação:

In [13]:
X_aval = catboost_encoder.transform(X_aval)

Realizando novas predições, mas agora em relação ao novo df de avaliação:

In [14]:
# Fazendo as previsões com a base de avaliação
rf_predictions = rf_model.predict_proba(X_aval)[:, 1] # Utilizando apenas o modelo que obteve melhor resultado anterior
#lr_predictions = lr_model.predict_proba(X_aval)[:, 1] 

Salvando dados para subir no desafio do Kaggle

In [15]:
# Transformando previsões em valores binários
rf_predictions= (rf_predictions >= 0.5).astype(int)

# Criando um DataFrame para salvar as previsões
df_predictions = pd.DataFrame({
    'PassengerId': X_aval_original['PassengerId'], 
    'Survived': rf_predictions
})

# Salvando em um arquivo CSV para upload no Kaggle
df_predictions.to_csv('predictions.csv', index=False) # Obtive 0.75119 como pontuação pública

# Refatoração 

## Cross Validation e Pipeline (Não implementado)

Nesta etapa, será definida as diferentes divisões da base de dados, além da implementação do encoding (evitando data leakage) para cada um dos dois algoritmos a serem implementados: RandomForest e LogisticRegression

In [16]:
# Separando inicialmente as variáveis preditoras e a target #pesquisar sobre arquivo .picle; metrica ks test (russa) muito utilizada em credito
X = df_model.drop(columns='Survived')
y = df_model['Survived']

# Configurando o Cross-Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Os pipelines serão criados com o CatBoost Encoder já embutido
# Criando o pipeline com Logistic Regression
pipeline_logreg = Pipeline([
    ('catboost_encoder', CatBoostEncoder(cols=col_categoricas, random_state=42)),  
    ('classifier', LogisticRegression(solver='liblinear', random_state=42))                           
])

# Criando o pipeline com RandomForestClassifier
pipeline_rf = Pipeline([
    ('catboost_encoder', CatBoostEncoder(cols=col_categoricas, random_state=42)), 
    ('classifier', RandomForestClassifier(random_state=42))                         
])

# Passo 6: Realizar o Cross-Validation e calcular as métricas (ROC-AUC)
scores_logreg = cross_val_score(pipeline_logreg, X, y, cv=cv, scoring='roc_auc') # crossvalidate pesquisar (retorna o melhor) https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html
scores_rf = cross_val_score(pipeline_rf, X, y, cv=cv, scoring='roc_auc')

# Passo 7: Mostrar os resultados
print(f'ROC-AUC para Regressão Logística em cada fold: {scores_logreg}')
print(f'ROC-AUC médio para Regressão Logística: {scores_logreg.mean()}')
print(f'ROC-AUC máximo para Regressão Logística: {scores_logreg.max()}')

print(f'\nROC-AUC para Random Forest em cada fold: {scores_rf}')
print(f'ROC-AUC médio para Random Forest: {scores_rf.mean()}')
print(f'ROC-AUC máximo para Random Forest: {scores_rf.max()}')

#pipeline_rf.predict_proba(X_test)k

ROC-AUC para Regressão Logística em cada fold: [0.85250329 0.82927807 0.82199198 0.82486631 0.87760936]
ROC-AUC médio para Regressão Logística: 0.8412498035802782
ROC-AUC máximo para Regressão Logística: 0.8776093604573859

ROC-AUC para Random Forest em cada fold: [0.88412385 0.87794118 0.82800802 0.85147059 0.86531046]
ROC-AUC médio para Random Forest: 0.861370819459524
ROC-AUC máximo para Random Forest: 0.8841238471673254
